In [1]:
#pip install --no-cache-dir jupyter langchain_openai langchain_community langchain langgraph faiss-cpu sentence-transformers ipywidgets transformers nltk scikit-learn matplotlib markdown langchain_chroma

import yaml
import ipywidgets as widgets
from IPython.display import display
import pickle
import markdown

from story_sage.story_sage import StorySage

In [2]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

api_key = config['OPENAI_API_KEY']
chroma_path = config['CHROMA_PATH']
chroma_collection = config['CHROMA_COLLECTION']
merged_characters_path = f'./characters/{chroma_collection}_characters.pkl'


with open(merged_characters_path, 'rb') as f:
    character_dict = pickle.load(f)

story_sage = StorySage(api_key=api_key, chroma_path=chroma_path, 
                       chroma_collection_name=chroma_collection,
                       character_dict=character_dict, n_chunks=15)

In [3]:
story_sage.invoke('who is dobby?')

['book number: 2, chapter: 2, excerpt: “Dobby has known it for months, sir. Harry Potter must not put himself in peril. He is too important, sir!” “What terrible things?” said Harry at once. “Who’s plotting them?” Dobby made a funny choking noise and then banged his head frantically against the wall. “All right!” cried Harry, grabbing the elf’s arm to stop him. “You can’t tell me.', 'book number: 2, chapter: 10, excerpt: “It was you!” he said slowly. “You stopped the barrier from letting us through!” “Indeed yes, sir,” said Dobby, nodding his head vigorously, ears flapping. “Dobby hid and watched for Harry Potter and sealed the gateway and Dobby had to iron his hands afterward” — he showed Harry ten long, bandaged fingers — “but Dobby didn’t care, sir, for he thought Harry Potter was safe, and never did Dobby dream that Harry Potter would get to school another way!” He was rocking backward and forward, shaking his ugly head. “Dobby was so shocked when he heard Harry Potter was back at 

("Dobby is a house-elf introduced in *Harry Potter and the Chamber of Secrets*. He is characterized by his loyalty, eagerness to help Harry Potter, and his desire for freedom from servitude. Here are some key points about Dobby based on the excerpts provided:\n\n- **House-Elf**: Dobby identifies himself as a house-elf bound to serve the Malfoy family until he is given clothes to be freed. He wears a filthy pillowcase as a mark of his enslavement (Chapter 10).\n  \n- **Protective Nature**: Dobby comes to warn Harry about the dangers he faces at Hogwarts and urges him not to return to school, believing that terrible things are going to happen there (Chapter 2, Chapter 10).\n\n- **Self-Punishment**: Dobby often punishes himself for perceived mistakes, demonstrating the harsh treatment house-elves endure from their masters. He’s seen banging his head against walls or other objects when he feels he has disobeyed (Chapter 2, Chapter 10).\n\n- **Admiration for Harry**: Dobby looks up to Harry

In [3]:
retriever = story_sage.retriever

chunks = retriever.retrieve_chunks('who is dobby', 10, 10, [])

In [4]:
chunks

{'ids': [['f9354699-638c-4384-83cf-63c05faa7934',
   'd1d0b44e-f272-4017-837c-71bb7790cf3b',
   'd94cf561-0888-4591-8437-4f13fefa61c8',
   'df4055be-3e7d-4bdf-9f35-93e8a90eb004',
   '7145dcc4-ccfc-4d9d-96e8-679d2f9ef4ab',
   '68933eaa-2405-4353-aa5b-cb63ea1a6884',
   '5a3c2d06-b11e-4366-9e13-7491f0b36f0b',
   '91be9f40-9702-4ad5-b632-e9ace8438bce',
   'e166236d-00d5-4971-99e9-2a59fe88dc61',
   '4fc6cb91-9c61-4095-880f-2243a24accd8',
   'a9179ea2-eace-4d69-aa1a-0aa1509750ba',
   '4764c960-c094-44a2-9b2c-33355eec5b60',
   '291f743d-670a-4295-8faa-e50e963c6ea6',
   '3f471bd5-15f6-44c1-8119-161924c9d5c1',
   '4dfb97d3-70dd-4c99-9b48-46094bedcf3d']],
 'embeddings': None,
 'documents': [['“Dobby has known it for months, sir. Harry Potter must not put himself in peril. He is too important, sir!” “What terrible things?” said Harry at once. “Who’s plotting them?” Dobby made a funny choking noise and then banged his head frantically against the wall. “All right!” cried Harry, grabbing the elf’s 

In [5]:
formatted_chunks = [
    f"book number: {meta['book_number']}, chapter: {meta['chapter_number']}, excerpt: {doc}"
    for meta, doc in zip(chunks['metadatas'][0], chunks['documents'][0])
]

formatted_chunks

['book number: 2, chapter: 2, excerpt: “Dobby has known it for months, sir. Harry Potter must not put himself in peril. He is too important, sir!” “What terrible things?” said Harry at once. “Who’s plotting them?” Dobby made a funny choking noise and then banged his head frantically against the wall. “All right!” cried Harry, grabbing the elf’s arm to stop him. “You can’t tell me.',
 'book number: 2, chapter: 10, excerpt: “It was you!” he said slowly. “You stopped the barrier from letting us through!” “Indeed yes, sir,” said Dobby, nodding his head vigorously, ears flapping. “Dobby hid and watched for Harry Potter and sealed the gateway and Dobby had to iron his hands afterward” — he showed Harry ten long, bandaged fingers — “but Dobby didn’t care, sir, for he thought Harry Potter was safe, and never did Dobby dream that Harry Potter would get to school another way!” He was rocking backward and forward, shaking his ugly head. “Dobby was so shocked when he heard Harry Potter was back at

In [3]:
# Define the input and output widgets
input_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    continuous_update=False,
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click to submit your question',
    icon='check'
)

book_number_box = widgets.IntText(
    value=10,
    description='Book Number:',
    disabled=False
)

chapter_number_box = widgets.IntText(
    value=None,
    description='Chapter Number:',
    disabled=False
)

status_box = widgets.Output(layout={'min_height': '50px'})
output_box = widgets.Output(layout={'min_height': '200px'})
context_box = widgets.Output(layout={'min_height': '200px'})

# Create a spinner widget
spinner = widgets.HTML(
    value="""<i class="fa fa-spinner fa-spin" style="font-size:24px; color:#2a9df4;"></i>""",
    placeholder='Loading...',
    description=''
)

# Make sure Font Awesome is available
display(widgets.HTML("<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css'>"))

def wrap_answer(answer):
    html = markdown.markdown(answer)
    return f"<div style='background-color: #f9f9f9; padding: 10px; border-radius: 5px;'>{html}</div>"

def show_results(answer, context):
    with output_box:
        output_box_contents = []
        output_box_contents.append("<h3>Answer</h3>")
        output_box_contents.append(wrap_answer(answer))
        display(widgets.HTML(''.join(output_box_contents)))
        with context_box:
            context_box.clear_output()
            context_box_contents = []
            context_box_contents.append("<h3>Context</h3>")
            for idx in range(len(context['metadatas'])):
                meta = context['metadatas'][0][idx]
                content = context['documents'][0][idx]
                context_box_contents.append(f"<p><strong>Book Number:</strong> {meta['book_number']} <strong>Chapter Number:</strong> {meta['chapter_number']}</p>")
                context_box_contents.append(f"<p>{content}</p>")
            display(widgets.HTML(wrap_answer("".join(context_box_contents))))

    

# Define the function to handle the button click
def submit_question(b):
    with status_box:
        status_box.clear_output()
        display(widgets.HTML(f"<h3>Retrieving top relevant chunks...</h3>"))
        with output_box:
            output_box.clear_output()
            display(spinner)
            answer, context = story_sage.invoke(
                question = input_box.value,
                book_number = book_number_box.value,
                chapter_number = chapter_number_box.value
            )
            output_box.clear_output()
    show_results(answer, context)

# Attach the handler to the button
submit_button._click_handlers.callbacks.clear()
submit_button.on_click(submit_question)

# Attach the handler to the input box for the return key
#input_box.observe(submit_question)

# Display the widgets
display(status_box, book_number_box, chapter_number_box, input_box, submit_button, output_box, context_box)

HTML(value="<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-aw…

Output(layout=Layout(min_height='50px'))

IntText(value=10, description='Book Number:')

IntText(value=0, description='Chapter Number:')

Text(value='', continuous_update=False, description='Question:', placeholder='Type your question here...')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit your question')

Output(layout=Layout(min_height='200px'))

Output(layout=Layout(min_height='200px'))

UnboundLocalError: cannot access local variable 'answer' where it is not associated with a value

In [4]:
story_sage.state

{}